# Corrigé du devoir 1

# Première partie

On interroge l’API _Découpage administratif communes_ du gouvernement français en envoyant une requête à l’adresse :
[https://geo.api.gouv.fr/communes?nom=Lille&boost=population&fields=population,surface](https://geo.api.gouv.fr/communes?nom=Lille&boost=population&fields=population,surface) pour récupérer les données géographiques (population et superficie) de la ville de Lille. La réponse de l’API est :

```python
[{"population": 234475, "surface": 3474.9, "nom": "Lille"},
 {"population": 9810, "surface": 2658.93, "nom": "Lillers"},
 {"population": 367, "surface": 379.17, "nom": "Lillemer"},
 {"population": 13151, "surface": 315.78, "nom": "Saint-André-lez-Lille"},
 {"population": 8797, "surface": 1464.34, "nom": "Lillebonne"},
 {"population": 10798, "surface": 484.71, "nom": "Marquette-lez-Lille"}]
```

**(1)** Comment s’appelle le format dans lequel les données sont reçues ?

Les données sont reçues au format JSON.

On suppose que la réponse de l’API est stockée dans une variable `reponse`.

**(2)** Quel est le type de la variable `reponse` ? de `reponse[0]` ? de `reponse[0]["surface"]` ?

`reponse` est un tableau de dictionnaires.

`reponse[0]` est un dictionnaire dont les clés sont `'population'`, `'surface'` et `'nom'`.

`reponse[0]["surface"]` est un flottant.

**(3)** Dans le contexte de l’API, à quoi correspond `len(reponse)` ?

`len(reponse)` correspond au nombre de villes dont le nom contient la chaîne `'Lille'`.

**(4)** Quelles expressions permettent :
-	d’accéder à la population de la ville de Lille ? `reponse[0]['population']`
-	d’accéder à la superficie de la ville de Lille ? `reponse[0]['surface']`
-	de calculer la densité de la ville de Lille ? `reponse[0]['population'] / reponse[0]['surface']`

**(5)**	À quelle adresse faut-il interroger l’API pour récupérer les données géographiques de la ville de Paris ?

À l'adresse [https://geo.api.gouv.fr/communes?nom=Paris&boost=population&fields=population,surface](https://geo.api.gouv.fr/communes?nom=Paris&boost=population&fields=population,surface).

## Exercice 2

On donne la définition d’une fonction `api_openfoodfacts` :

```python
def api_openfoodfacts(code_barres):
    url = f"https://world.openfoodfacts.org/api/v0/product/{code_barres}.json"
    reponse = requests.get(url)
    reponse = reponse.json()
    if reponse["status"] == 1:
        return {"nom": reponse["product"]["product_name"],
                "nutri_score": reponse["product"]["nutrition_grade_fr"]}
    else:
        return {"nom": "Produit inconnu",
                "nutri_score": None}
```

Si on interroge l’API Open Food Facts à partir d’un code-barres erroné, on obtient une réponse du type :

```python
{"code": "8076800376998", "status": 0, "status_verbose": "product not found"}
```

**(1)** Dire, en justifiant, ce que renvoie la fonction `api_openfoodfacts` dans le cas où on l’appelle avec un code-barres erroné en argument.

Dans le cas où l'API est interrogée avec un code-barres erroné, elle renvoie des données JSON avec un `'status'` égal à `0`. Dans ce cas, l'instruction conditionnelle présente dans la définition de la fonction `api_openfoodfacts` fait que cette fonction renvoie le dictionnaire :
```python
{"nom": "Produit inconnu", "nutri_score": None}
```

L’appel `api_openfoodfacts(3166352967037)` renvoie ceci :

```python
{"nom": "Poisson à l'andalouse riz safrané et courgettes grillées",
 "nutri_score": "a"}
```

**(2)**	Écrire la spécification de la fonction `api_openfoodfacts`.

```python
"""
Interroge l'API Open Food Facts et renvoie le nom et le nutriscore d'un produit alimentaire à partir de son code-barres.
- Entrée : code_barres (chaîne de caractères ou entier)
- Sortie : (dictionnaire dont les clés sont "nom" et "nutri_score")
"""
```

**(3)**	Définir une fonction `nutriscore` qui :
-	prend en paramètre d'entrée un tableau `tab_codes` contenant des numéros de code-barres, et qui
-	renvoie le dictionnaire qui associe à chacune des chaînes `"a"`, `"b"`, `"c"`, `"d"` et `"e"` la liste des noms des produits au nutriscore correspondant.

`nutriscore([3166352967037, 3497917000495, 3560070682683, 3245412511561, 3046920022651])` doit par exemple renvoyer :

```python
{"a": ["Poisson à l'andalouse riz safrané et courgettes grillées"],
 "b": [],
 "c": ["Chips saveur Indian curry", "Yaourt à la GRECQUE NATURE"],
 "d": [],
 "e": ["Biscuits chocolat au lait", "Lindt Chocolate Excellence"]}
```

```python
def nutriscore(tab):
    dico_noms = {cle: [] for cle in "abcde"}
    for code_barres in tab:
        dico = api_openfoodfacts(code_barres)
        dico_noms[dico["nutri_score"]]. append(dico["nom"])
    return dico_noms
```

# Seconde partie

**Les documents et la recherche d'informations sur internet sont strictement interdits.**

## Exercice 1 - Championnat de France de rugby

L'API _SportRadar_ permet de récupérer des données sur les compétitions sportives, par exemple le championnat de France de rugby (Top 14).

On donne la fonction `resultats_top14` :

In [1]:
import requests

def resultats_top14(jour, mois, annee):
    """
    Interroge l'API SportRadar et récupère les résultats des matchs de Top 14 pour une date donnée.
    - Entrées : jour, mois, annee (chaînes de caractères aux formats JJ, MM et AAAA)
    - Sortie : donnees (tableau dont les élément sont des dictionnaires)
    """
    url = f"https://api.sportradar.us/rugby/trial/v2/union/en/schedules/{annee}-{mois}-{jour}/results.json?api_key=kcfszdfgk23h6cfxhjght7su"
    reponse = requests.get(url)
    reponse = reponse.json()
    donnees = []
    for match in reponse['results']:
        if match['sport_event']['sport_event_context']['competition']['name'] == 'Top 14':
            if not match['sport_event_status']['status'] == 'postponed':
                dom = match['sport_event']['competitors'][0]['name']
                ext = match['sport_event']['competitors'][1]['name']
                score_dom = match['sport_event_status']['home_score']
                score_ext = match['sport_event_status']['away_score']
                donnees.append({'dom' : dom,
                                'ext' : ext,
                                'score_dom' : score_dom,
                                'score_ext' : score_ext})
    return donnees

**(1)** Après avoir lu attentivement la spécification de la fonction, expliquer pourquoi l'exécution de la cellule suivante provoque une erreur.

In [2]:
tab_matchs = resultats_top14(17, 9, 2022)

KeyError: 'results'

Les arguments passés en paramètre de la fonction doivent être trois chaînes de caractères au format `'JJ'`, `'MM'` et `AAAA`. Il faut donc écrire :

In [3]:
tab_matchs = resultats_top14("17", "09", "2022")

**(2)** Ecrire des lignes de code permettant d'afficher le nombre de matchs de Top 14 qui ont été joués le samedi 17 septembre 2022.

In [4]:
print(len(tab_matchs))

6


**(3)** Quelles sont les clés du dictionnaire `tab_matchs[0]` ? Préciser quel est le type des valeurs associées à chaque clé.

In [5]:
print(tab_matchs[0])

{'dom': 'Union Bordeaux Begles', 'ext': 'Castres Olympique', 'score_dom': 33, 'score_ext': 12}


Les clés sont `'dom'` (équipe jouant à domicile), `'ext'` (équipe jouant à l'extérieur), `'score_dom'` (score de l'équipe à domicile) et `'score_ext'` (score de l'équipe à l'extérieur).

**(4)** Ecrire des lignes de code permettant d'afficher les scores des matchs de Top 14 joués le 17 septembre 2022 au format :

`Union Bordeaux Begles 33 - 12 Castres Olympique`.

In [6]:
for match in tab_matchs:
    print(f"{match['dom']} {match['score_dom']} - {match['score_ext']} {match['ext']}")

Union Bordeaux Begles 33 - 12 Castres Olympique
Racing 92 32 - 19 Lyon Ou
Stade Rochelais 43 - 8 USA Perpignan
Stade Francais Paris 26 - 16 Aviron Bayonne
AC Brive 26 - 31 Montpellier Herault Rugby
Section Paloise 26 - 16 Stade Toulousain


**(5)** Sachant que le seul match de Top 14 joué le 24 juin 2022 était la finale de la compétition, écrire des lignes de code permettant d'afficher le nom du champion de France en titre.

In [7]:
finale = resultats_top14("24", "06", "2022")[0]
if finale['score_dom'] > finale['score_ext']:
    print(f"Le champion est {finale['dom']}.")
elif finale['score_ext'] > finale['score_dom']:
    print(f"Le champion est {finale['ext']}.")
else:
    print("Match nul")

Le champion est Montpellier Herault Rugby.


## Exercice 2 - Planètes du système solaire

L'API _Système solaire_ permet de récupérer des données sur les planètes du système solaire et sur leurs satellites naturels. On interroge l'API de la façon suivante :

In [8]:
import requests

In [9]:
url = f"https://api.le-systeme-solaire.net/rest/bodies"
reponse = requests.get(url)
reponse = reponse.json()

Voici une partie de la réponse reçue :

In [10]:
reponse["bodies"][0]

{'id': 'lune',
 'name': 'La Lune',
 'englishName': 'Moon',
 'isPlanet': False,
 'moons': None,
 'semimajorAxis': 384400,
 'perihelion': 363300,
 'aphelion': 405500,
 'eccentricity': 0.0549,
 'inclination': 5.145,
 'mass': {'massValue': 7.346, 'massExponent': 22},
 'vol': {'volValue': 2.1968, 'volExponent': 10},
 'density': 3.344,
 'gravity': 1.62,
 'escape': 2380.0,
 'meanRadius': 1737.0,
 'equaRadius': 1738.1,
 'polarRadius': 1736.0,
 'flattening': 0.0012,
 'dimension': '',
 'sideralOrbit': 27.3217,
 'sideralRotation': 655.728,
 'aroundPlanet': {'planet': 'terre',
  'rel': 'https://api.le-systeme-solaire.net/rest/bodies/terre'},
 'discoveredBy': '',
 'discoveryDate': '',
 'alternativeName': '',
 'axialTilt': 6.68,
 'avgTemp': 0,
 'mainAnomaly': 0.0,
 'argPeriapsis': 0.0,
 'longAscNode': 0.0,
 'bodyType': 'Moon',
 'rel': 'https://api.le-systeme-solaire.net/rest/bodies/lune'}

**(1)** Quel est le type de `reponse["bodies"][0]["isPlanet"]` ?

Il s'agit d'un booléen.

**(2)** Ecrire des lignes de code permettant d'afficher uniquement le nom des huit planètes du système solaire.

In [11]:
for astre in reponse["bodies"]:
    if astre["isPlanet"]:
        print(astre["name"])

Uranus
Neptune
Jupiter
Mars
Mercure
Saturne
La Terre
Vénus


**(3)** Sachant que la masse de la Lune est d'environ $7,3 \times 10^{22}$ kilogrammes et que son volume est d'environ $2,2 \times 10^{10}$ kilomètres cubes, écrire des lignes de code permettant d'afficher, pour chacune des huit planètes du système solaire, sa masse en milliards de tonnes et son volume en kilomètres cubes.

In [12]:
for astre in reponse["bodies"]:
    if astre["isPlanet"]:
        nom = astre["name"]
        masse = int(astre["mass"]["massValue"] * 10**(astre["mass"]["massExponent"]-12))
        volume = int(astre["vol"]["volValue"] * 10**astre["vol"]["volExponent"])
        print(f"{nom} : {masse} milliards de tonnes, {volume} kilomètres cubes")

Uranus : 86812700000000 milliards de tonnes, 68330000000000 kilomètres cubes
Neptune : 102413000000000 milliards de tonnes, 62539999999999 kilomètres cubes
Jupiter : 1898190000000000 milliards de tonnes, 1431280000000000 kilomètres cubes
Mars : 641712000000 milliards de tonnes, 163180000000 kilomètres cubes
Mercure : 330114000000 milliards de tonnes, 60830000000 kilomètres cubes
Saturne : 568336000000000 milliards de tonnes, 827130000000000 kilomètres cubes
La Terre : 5972370000000 milliards de tonnes, 1083210000000 kilomètres cubes
Vénus : 4867470000000 milliards de tonnes, 928430000000 kilomètres cubes


**(4)** La densité d'une planète peut se calculer en divisant sa masse (en milliards de tonnes) par son volume (en kilomètres cubes). Ecrire des lignes de code permettant d'afficher le nom de la planète du système solaire dont la densité est la plus élevée.

In [13]:
densite_max = 0
nom_densite_max = ""
for astre in reponse["bodies"]:
    if astre["isPlanet"]:
        nom = astre["name"]
        masse = int(astre["mass"]["massValue"] * 10**(astre["mass"]["massExponent"]-12))
        volume = int(astre["vol"]["volValue"] * 10**astre["vol"]["volExponent"])
        densite = masse / volume
        if densite > densite_max:
            densite_max = densite
            nom_densite_max = astre["name"]
print(nom_densite_max)

La Terre


## Exercice 3 - Election présidentielle française de 2012

 L'envoi d'une requête GET à l'adresse [https://ntoulzac.github.io/Cours-NSI-Terminale/api/donnees/presidentielle_2012_tour2.json](https://ntoulzac.github.io/Cours-NSI-Terminale/api/donnees/presidentielle_2012_tour2.json) permet de récupérer, département par département, les résultats du second tour de l'élection présidentielle de 2012.

**(1)** Ecrire des lignes de code permettant de récupérer dans une variable `reponse` la réponse renvoyée par l'API sous la forme d'un dictionnaire.

In [14]:
import requests
reponse = requests.get("https://ntoulzac.github.io/Cours-NSI-Terminale/api/donnees/presidentielle_2012_tour2.json")
reponse = reponse.json()

**(2)** Quel est le type de `reponse['couleurs']` ? de `reponse['tour2']` ? de `reponse['tour2'][0]` ?

`reponse['couleurs']` est un dictionnaire dont les clés sont les noms des deux candidats au second tour de la présidentielle 2012. Les valeurs associées sont des chaînes de caractères correspondant à des couleurs au format RVB.

`reponse['tour2']` est un tableau de dictionnaires.

`reponse['couleurs'][0]` est un dictionnaire correspondant aux résultats du second tour de la présidentielle pour un département.

**(3)** Compléter la définition de la fonction `carte_resultats_2012` conformément à sa spécification.

In [15]:
from carte_france import creer_svg_departements

def couleurs_departements():
    """
    Détermine pour chaque département la couleur du candidat arrivé en tête au second tour en 2012.
    - Sortie : resultats (dictionnaire dont les clés sont des numéros de département et les valeurs
               associées sont le code couleur du candidat arrivé en tête au second tour en 2022)
    """
    url = "https://ntoulzac.github.io/Cours-NSI-Terminale/api/donnees/presidentielle_2012_tour2.json"
    reponse = requests.get(url)
    reponse = reponse.json()
    COULEURS = reponse["couleurs"]
    resultats = {dep["dep"]: COULEURS[dep["candidats"][0]["nom"]] for dep in reponse["tour2"]}
    return resultats

La cellule suivante permet d'afficher quel candidat est arrivé en tête dans chaque département.

In [16]:
creer_svg_departements(couleurs_departements(), f"candidat_en_tete_2012.svg", True)

_En rose, François Hollande. En bleu, Nicolas Sarkozy._